# Scissors scRNA-Seq

**Created**: 18 April 2022

## Environment

In [1]:
if (!requireNamespace("Scissors", quietly=TRUE)) {
    devtools::install_github("sunduanchen/Scissor")
}

if (!requireNamespace("reticulate", quietly=TRUE)) {
    install.packages("reticulate")
}

if (!requireNamespace("lubridate", quietly=TRUE)) {
    install.packages("lubridate")
}

Skipping install of 'Scissor' from a github remote, the SHA1 (311560a1) has not changed since last install.
  Use `force = TRUE` to force installation



In [43]:
library(reticulate)
library(Seurat)
library(SeuratDisk)
library(Scissor)
library(tidyverse)
library(lubridate)

setwd("~/eQTL_pQTL_Characterization/")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

## Load Data

This data is from COMBAT, which is a recent compendium of single cell RNA-seq performed on peripheral blood mononuclear cells (PBMCs). The data contains healthy donors and septic patients.

In [3]:
scanpy <- reticulate::import("scanpy")

In [4]:
combat.annot <- scanpy$read_h5ad("/nfs/users/nfs_n/nm18/gains_team282/nikhil/data/COMBAT/COMBAT-CITESeq-EXPRESSION-ATLAS.h5ad")

In [30]:
expression <- t(combat.annot$X)
colnames(expression) <- combat.annot$obs_names$to_list()
rownames(expression) <- combat.annot$var_names$to_list()

Filter to only include gene expression features and convert to a matrix that can be processed by Seurat.

In [31]:
expression <- expression[combat.annot$var$feature_types == "Gene Expression",]
rownames(expression) <- combat.annot$var$gene_ids[combat.annot$var$feature_types == "Gene Expression"]
expression <- as(expression, "CsparseMatrix")

Default pre-processing recommended by Scissor before using the Seurat object.

In [32]:
combat <- CreateSeuratObject(expression, min.cells=400, min.features=0)
combat <- NormalizeData(combat, normalization.method="LogNormalize", scale.factor=10000, verbose=F)
combat <- FindVariableFeatures(combat, selection.method="vst", verbose=F)
combat <- ScaleData(combat, verbose=F)
combat <- RunPCA(combat, features=VariableFeatures(combat), verbose=F)
combat <- FindNeighbors(combat, dims=1:10, verbose=F)
combat <- AddMetaData(combat, combat.annot$obs)

Warning message:
“The new data doesn't have the same number of features as the current data”
Warning message:
“Adding features not currently present in the object”


In [ ]:
rm(combat.annot)

In [33]:
gene.exp <- read.table("/lustre/scratch119/humgen/projects/gains_team282/eqtl/data/logcpm_864_20412_hla.txt")
sample.info <- read.table("/nfs/team282/data/gains_team282/Sample_info_864.txt")
sample.key <- read.table("/nfs/team282/data/gains_team282/Sample_key.txt", header=T)

colnames(gene.exp) <- gsub("^GA", "", colnames(gene.exp))

sample.info <- sample.info %>%
    dplyr::mutate(supplier_name=gsub("^GA", "", supplier_name)) %>%
    dplyr::filter(supplier_name %in% colnames(gene.exp)) %>%
    dplyr::mutate(Time.Point=as.numeric(gsub("^.*_", "", supplier_name)))

rownames(sample.info) <- sample.info$supplier_name
sample.info <- sample.info[colnames(gene.exp),]

In [34]:
head(gene.exp)

,UK02270173_3,UK15130120_3,UK58000006_3,UK47010004_3,UK42020088_5,UK47490007_3,UK02770164_3,UK02770164_5,UK02630151_3,UK42150107_1,⋯,UK59070043_3,UK59070043_5,UK02510223_3,UK02XX0336_5,UK29090086_3,UK02XX0335_1,UK02XX0334_3,UK01210130_3,UK01210130_5,UK01380125_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000230521,0.6644339,0.58271604,0.4046756,0.20905157,0.2211571,0.4288519,0.5133516,0.57768877,0.6990766,0.3812037,⋯,0.3823184,0.5423684,0.86762630,0.73833461,0.6461583,0.1227063,0.9621386,0.65838459,1.03561474,0.4457301
ENSG00000225864,0.9817872,0.43879936,0.6022344,0.92340489,0.2211571,1.1450686,0.1921362,0.57768877,0.1974851,1.2290495,⋯,0.6842527,0.8539227,0.91886777,0.12581620,0.5511865,0.4384007,0.5465139,0.40803066,0.46899151,0.3544115
ENSG00000227766,1.2117156,0.30682669,0.6625010,0.79963286,0.6445686,0.3130290,0.1183231,0.16750450,0.1505900,1.0533199,⋯,0.4901384,0.4340781,0.37504164,0.04316943,0.6151905,0.3406488,0.3147798,0.43511377,0.17382742,0.1157514
ENSG00000237669,0.6420763,0.69256830,0.9792012,0.69929584,0.3765156,1.0756817,0.8912050,0.60129641,0.8534668,1.0533199,⋯,1.0790156,0.6102946,1.03953331,0.97965846,0.7061683,0.7489857,0.7919934,0.85168813,0.69120049,1.0798771
ENSG00000271581,2.4744895,1.33153140,1.8310464,2.47531701,1.8812042,2.0531366,0.5972544,1.00686213,1.2811361,2.4244938,⋯,2.4728831,1.5338898,1.59080138,0.62923925,1.2804253,1.4727511,1.2187650,1.46471386,1.44139443,1.1896250
ENSG00000285647,0.0000000,0.03379072,0.0000000,0.05517067,0.6748270,0.0000000,0.0000000,0.03509293,0.6990766,1.6556266,⋯,3.4117627,2.4012963,0.04682116,0.00000000,0.0000000,0.4695662,0.3762952,0.07078527,0.07206309,0.4159282


In [35]:
head(sample.info)

,manual_qc,id_study_lims,name,description,sanger_sample_id,supplier_name,last_updated,id_library_lims,id_pool_lims,id_iseq_flowcell_tmp,⋯,TIN.median.,TIN.stdev.,Concentration,num_samples_per_lane,SRSUnsup,globin_rate_fromcounts,PCOutlier,OtherOutlier,GAinSID,Time.Point
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<lgl>,<lgl>,<chr>,<dbl>
UK02270173_3,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032857,UK02270173_3,2019-09-30 21:01:38,DN548016D,NT1557159B,6930558,⋯,76.82800,23.72743,116.731,77,2,0.6790401,FALSE,FALSE,UK02270173,3
UK15130120_3,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032858,UK15130120_3,2019-09-30 21:01:38,DN548016D,NT1557159B,6930559,⋯,71.49895,24.13243,76.185,77,2,0.6826886,FALSE,FALSE,UK15130120,3
UK58000006_3,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032859,UK58000006_3,2019-09-30 21:01:38,DN548016D,NT1557159B,6930560,⋯,78.07773,23.24183,59.767,77,2,1.4290063,FALSE,FALSE,UK58000006,3
UK47010004_3,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032860,UK47010004_3,2019-09-30 21:01:38,DN548016D,NT1557159B,6930561,⋯,75.05450,23.56361,105.000,77,1,0.9177558,FALSE,FALSE,UK47010004,3
UK42020088_5,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032861,UK42020088_5,2019-09-30 21:01:38,DN548016D,NT1557159B,6930562,⋯,72.53810,26.03766,87.843,77,1,0.4031685,FALSE,FALSE,UK42020088,5
UK47490007_3,1,5890,HG_The Genomic Advances in Sepsis (GAinS) RNA-seq,Total leukocyte RNA,gains8032862,UK47490007_3,2019-09-30 21:01:38,DN548016D,NT1557159B,6930563,⋯,68.60305,24.27960,67.484,77,2,1.1993500,FALSE,FALSE,UK47490007,3


In [36]:
outcome <- read.table("/nfs/team282/data/gains_team282/ClinicalData/clinical_data_tsv/OUT_12jun2019.tsv", sep="\t", header=T, quote="")

outcome <- outcome %>%
    dplyr::mutate(GAinSID=stringr::str_to_upper(SubjectBarCode)) %>%
    dplyr::mutate(GAinSID=gsub("^GA", "", GAinSID))

rownames(outcome) <- outcome$GAinSID

In [37]:
head(outcome)

,CenterNumber,SubjectNumber,diagnosis,study,SubjectBarCode,DICU,alivedead,dhospdis,aldead,at6Malived,⋯,failure3,unrelated3,persist3,unrelatc3,unreldesc3,AtLeastOneCause,EDTAsent,DhospICU,Birthdate,GAinSID
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
UK01310070,UK001,2,1,ALL SUBJECT,uk01310070,1/15/08,1,4/16/08,2,2,⋯,0,0,1,0,,1,1,12/28/07,6/15/35,UK01310070
UK01150069,UK001,3,2,ALL SUBJECT,uk01150069,12/17/07,1,12/17/07,1,1,⋯,0,0,0,0,,0,1,12/14/07,9/26/26,UK01150069
UK01470071,UK001,5,1,ALL SUBJECT,UK01470071,1/30/08,1,4/20/08,1,1,⋯,0,0,0,0,,0,1,1/4/08,8/17/34,UK01470071
UK01630072,UK001,6,1,ALL SUBJECT,UK01630072,1/7/08,1,2/13/08,1,1,⋯,0,0,0,0,,0,1,1/5/08,1/20/68,UK01630072
UK01950074,UK001,8,1,ALL SUBJECT,UK01950074,1/14/08,1,4/16/08,1,1,⋯,0,0,0,0,,0,1,1/9/08,2/4/40,UK01950074
UK01140075,UK001,9,1,ALL SUBJECT,UK01140075,1/28/08,2,,NA,,⋯,0,0,0,0,,1,1,1/28/08,11/1/29,UK01140075


## Subset Patients

The same patients have contributed up to three samples to the data (D1, D3, and/or D5). I have decided to choose the last time point available for each patient.

In [38]:
sample.map <- sample.info %>%
    dplyr::filter(GAinSID %in% outcome$GAinSID) %>%
    dplyr::group_by(GAinSID) %>%
    dplyr::summarize(Last.Time.Point=max(Time.Point)) %>%
    dplyr::mutate(Sample=paste0(GAinSID, "_", Last.Time.Point))

gene.exp <- as.matrix(gene.exp[,sample.map$Sample])

cox.outcomes <- outcome[sample.map$GAinSID,] %>%
    dplyr::mutate(Date.Hospitalized=lubridate::mdy(DhospICU), Date.Death=lubridate::mdy(M6ddeath)) %>%
    dplyr::mutate(Endpoint=as.duration(Date.Hospitalized %--% Date.Death) / lubridate::ddays(1)) %>%
    dplyr::mutate(Endpoint=replace(Endpoint, Endpoint > 28, NA)) %>%
    dplyr::mutate(Status=ifelse(is.na(Endpoint), 0, 1)) %>%
    dplyr::select(Endpoint, Status) %>%
    dplyr::mutate(Endpoint=replace(Endpoint, is.na(Endpoint), 28)) %>%
    dplyr::select(time=Endpoint, status=Status)

In [12]:
BiocManager::install("preprocessCore", configure.args="--disable-threading", force = TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.12 (BiocManager 1.30.16), R 4.0.5 (2021-03-31)

Installing package(s) 'preprocessCore'

Old packages: 'blob', 'brew', 'broom', 'Cairo', 'caret', 'circlize', 'clipr',
  'cluster', 'conquer', 'DEoptimR', 'dequer', 'desc', 'doParallel', 'foreach',
  'formatR', 'gdtools', 'gert', 'gower', 'haven', 'km.ci', 'lme4', 'maptools',
  'MASS', 'Matrix', 'mclogit', 'mgcv', 'nlme', 'nloptr', 'pbdZMQ', 'polynom',
  'pracma', 'processx', 'quantreg', 'RcppGSL', 'RCurl', 'readr', 'readxl',
  'recipes', 'reshape', 'Rfast', 'robust', 'robustbase', 'rrcov', 'RSQLite',
  'survival', 'survMisc', 'svglite', 'systemfonts', 'testthat', 'tzdb', 'umap',
  'uuid', 'waldo', 'WGCNA', 'XML'



In [42]:
infos1 <- Scissor(gene.exp, combat, cox.outcomes, alpha = 0.05, family = "cox")

ERROR: Error in asMethod(object): Cholmod error 'problem too large' at file ../Core/cholmod_dense.c, line 102


In [29]:
str(infos1)

List of 4
 $ para       :List of 3
  ..$ alpha : num 0.05
  ..$ lambda: num 0.01
  ..$ family: chr "cox"
 $ Coefs      : num [1:5000] 0 0 0 0 0 0 0 0 0 0 ...
 $ Scissor_pos: chr(0) 
 $ Scissor_neg: chr [1:713] "CTCTGGTTCGTACGGC-42" "CATATTCTCTGGAGCC-11" "CTAGTGAGTTCAGGCC-44" "GAACGGAAGGGTGTGT-12" ...


In [39]:
freqs <- table(pbmc[["cell_type"]][idx,]) / table(pbmc[["cell_type"]])
freqs[freqs > (5000 / ncol(pbmc))]


                     CD4-positive, alpha-beta T cell 
                                         0.007761966 
      central memory CD4-positive, alpha-beta T cell 
                                         0.004086505 
                           double negative thymocyte 
                                         0.007127193 
     effector memory CD4-positive, alpha-beta T cell 
                                         0.004254502 
                        hematopoietic precursor cell 
                                         0.005518764 
                                innate lymphoid cell 
                                         0.006756757 
                                       memory B cell 
                                         0.004200569 
naive thymus-derived CD8-positive, alpha-beta T cell 
                                         0.004149378 
                                 natural killer cell 
                                         0.004141075 
                   peripher